### Teste do servidor FastAPI

In [1]:
# Enviar mensagem para o servidor

message = "Me fale sobre tecnologia"

import requests
import datetime

url = "http://127.0.0.1:8001/message"

payload = {
    "phone_number": "5582999464789",
    "message": message,
    "message_id": "abc123",
    "user_name": "Teste",
    "timestamp": datetime.datetime.now().isoformat()
}

response = requests.post(url, json=payload)

print("Status code:", response.status_code)
print("Resposta:", response.json())

Status code: 200
Resposta: {'status': 'success', 'message': 'Mensagem adicionada para processamento em background'}


In [17]:
# Enviar mensagem para base de conhecimento

message = "Olá, meu nome é Marcelo, que horas são?"

import requests
import datetime

url = "http://127.0.0.1:8000/add-knowledge"

payload = {
    "phone_number": "5582999464789",
    "message": message,
    "message_id": "abc123",
    "user_name": "Teste",
    "timestamp": datetime.datetime.now().isoformat()
}

response = requests.post(url, json=payload)

print("Status code:", response.status_code)
print("Resposta:", response.json())

Status code: 401
Resposta: {'message': 'Unauthorized'}


### QDRANT

In [22]:
# Conectar ao Qdrant
from qdrant_client import QdrantClient
import requests

# 1. Verificar se Qdrant está rodando
try:
    response = requests.get("http://localhost:6333/collections")
    print("✅ Qdrant está rodando!")
except:
    print("❌ Qdrant não está rodando. Execute: docker-compose up -d")
    exit()

# Verificar se está conectado
client = QdrantClient(host="localhost", port=6333)
print("✅ Conectado ao Qdrant!")

# Ver todas as coleções
try:
    collections = client.get_collections()
    print("Coleções disponíveis:")
    for collection in collections.collections:
        print(f"  - {collection.name}: {client.count(collection.name).count} documentos")
except Exception as e:
    print(f"❌ Erro ao obter coleções: {e}")

print("✅ Coleções obtidas com sucesso!")

# Ver detalhes da coleção 'knowledge_base'
try:
    collection_info = client.get_collection('knowledge_base')
    print(f"Nome: {collection_info.name}")
    print(f"Pontos: {collection_info.points_count}")
    print(f"Vetores: {collection_info.vectors_count}")
    print(f"Status: {collection_info.status}")
except Exception as e:
    print(f"Erro ao obter detalhes da knowledge_base: {e}")

✅ Qdrant está rodando!
✅ Conectado ao Qdrant!
Coleções disponíveis:
  - knowledge_base: 5 documentos
✅ Coleções obtidas com sucesso!
Erro ao obter detalhes da knowledge_base: 'CollectionInfo' object has no attribute 'name'


In [25]:
from qdrant_client import QdrantClient
client = QdrantClient(location="localhost:6333")

collection_name = 'knowledge_base'

try:
    # Verificar se existe
    info = client.get_collection(collection_name)
    print(f"📊 Collection {collection_name} tem {info.points_count} pontos")
    
    client.delete_collection(collection_name)
    print("✅ Collection apagada!")
        
    # Recriar vazia
    from qdrant_client.http.models import Distance, VectorParams
    vectors_config = VectorParams(size=384, distance=Distance.COSINE)
    client.create_collection(collection_name=collection_name, vectors_config=vectors_config)
    print("✅ Collection recriada vazia!")
except Exception as e:
    print(f"❌ Erro: {e}")

📊 Collection knowledge_base tem 0 pontos
✅ Collection apagada!
✅ Collection recriada vazia!


In [ ]:
# Buscar todos os pontos (limitado a 100 por padrão)
points, next_page_offset = client.scroll(
    collection_name='knowledge_base',
    limit=100,
    with_payload=True,
    with_vector=False
)

print(f"Encontrados {len(points)} documentos:")
for i, point in enumerate(points, 1):
    print(f"\n--- Documento {i} ---")
    print(f"ID: {point.id}")
    print(f"Documento: {point.payload.get('document', 'N/A')}")
    print(f"Fonte: {point.payload.get('source', 'N/A')}")
    print(f"Adicionado em: {point.payload.get('added_at', 'N/A')}")

# Ver apenas metadados
points, _ = client.scroll(
    collection_name='knowledge_base',
    limit=100,
    with_payload=True,
    with_vector=False
)

print("Metadados dos documentos:")
for point in points:
    print(f"ID: {point.id}")
    print(f"  Fonte: {point.payload.get('source')}")
    print(f"  Adicionado: {point.payload.get('added_at')}")
    print("---")

In [ ]:
# Exemplo de busca vetorial (você precisará gerar um embedding)
from sentence_transformers import SentenceTransformer

# Carregar modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Texto para buscar
query_text = "sua consulta aqui"
query_vector = model.encode([query_text])[0].tolist()

# Buscar documentos similares
search_results = client.search(
    collection_name='knowledge_base',
    query_vector=query_vector,
    limit=5,
    with_payload=True
)

print(f"Resultados para: '{query_text}'")
for i, result in enumerate(search_results, 1):
    print(f"\n--- Resultado {i} (Score: {result.score:.4f}) ---")
    print(f"ID: {result.id}")
    print(f"Documento: {result.payload.get('document')}")

### SUPABASE

In [ ]:
# Teste do Supabase
from supabase import create_client
import os
from dotenv import load_dotenv

load_dotenv(".env")

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_ANON_KEY")

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

data = {
    "phone_number": "11999999999",
    "user_name": "Teste",
    "status": "active"
}
result = supabase.table("conversations").insert(data).execute()
print(result)

In [ ]:
# SELECT * FROM conversations LIMIT 1
result = supabase.table("conversations").select("*").limit(1).execute()
print(result.data)

### OLLAMA

In [ ]:
# Teste o Modelo
import os
from dotenv import load_dotenv
import requests

load_dotenv(".env")

OLLAMA_URL = os.getenv("OLLAMA_URL")

response = requests.post(
    f"{OLLAMA_URL}/api/generate",
    json={
        "model": "llama3.2",
        "prompt": "Olá, quem é você?"
    }
)
print(response)

In [ ]:
# CUDA

import torch
print(torch.cuda.is_available())  # Deve retornar True
#print(torch.cuda.current_device())  # Deve retornar 0 (ou outro índice, se múltiplas GPUs)
print(torch.cuda.get_device_name(0))  # Nome da sua GPU

In [ ]:
#Script de teste para o sistema RAG

import asyncio
import logging
from src.services.rag_system import RAGSystem

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

async def test_rag_system():
    """Testa o sistema RAG"""
    try:
        rag = RAGSystem()
        
        # Teste 1: Verificar conexão com Qdrant
        logger.info("=== Teste 1: Verificar conexão com Qdrant ===")
        rag._initialize_qdrant()
        
        if not rag.qdrant:
            logger.error("Falha na conexão com Qdrant")
            return
        
        # Teste 2: Verificar coleções
        logger.info("=== Teste 2: Verificar coleções ===")
        collections = rag.qdrant.get_collections()
        logger.info(f"Coleções disponíveis: {[c.name for c in collections.collections]}")
        
        # Teste 3: Adicionar documentos de teste
        logger.info("=== Teste 3: Adicionar documentos de teste ===")
        test_documents = [
            "O produto X custa R$ 100,00 e tem garantia de 1 ano.",
            "Para suporte técnico, entre em contato pelo WhatsApp ou email.",
            "Nossos horários de atendimento são de segunda a sexta, das 8h às 18h.",
            "Frete grátis para compras acima de R$ 200,00.",
            "Aceitamos cartão de crédito, PIX e boleto bancário."
        ]
        
        await rag.add_documents_to_rag(test_documents, [{"source": "teste"} for _ in test_documents])
        
        # Teste 4: Testar busca
        logger.info("=== Teste 4: Testar busca ===")
        query = "Quanto custa o produto X?"
        context = await rag.retrieve_context(query)
        logger.info(f"Query: {query}")
        logger.info(f"Contexto encontrado: {context}")
        
        # Teste 5: Testar busca com histórico
        logger.info("=== Teste 5: Testar busca com histórico ===")
        history = [
            {"message": "Olá, preciso de ajuda", "message_type": "incoming"},
            {"message": "Como posso ajudar?", "message_type": "outgoing"}
        ]
        context_with_history = await rag.retrieve_context("preços", history)
        logger.info(f"Query com histórico: preços")
        logger.info(f"Contexto encontrado: {context_with_history}")
        
        logger.info("=== Todos os testes concluídos com sucesso! ===")
        
    except Exception as e:
        logger.error(f"Erro durante os testes: {e}")
        import traceback
        logger.error(f"Traceback: {traceback.format_exc()}")

if __name__ == "__main__":
    asyncio.run(test_rag_system()) 

### PUB/SUB

In [14]:
# Publicar mensagem diretamente no PubSub

import os
from google.cloud import pubsub_v1

# Configurar credenciais e variáveis
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Projetos\python-llama-rag-wts\credenciais.json"
PROJECT_ID = os.getenv("GCP_PROJECT_ID", "treinamentoia-465816")
PUBSUB_TOPIC_ID = os.getenv("PUBSUB_TOPIC_ID", "whatsapp-messages")
SUBSCRIPTION_ID = os.getenv("PUBSUB_SUBSCRIPTION_ID", "whatsapp-messages-sub")

In [ ]:
# Publicar mensagem diretamente no PubSub
publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(PROJECT_ID, PUBSUB_TOPIC_ID)

data = "Mensagem de teste do PubSub".encode("utf-8")
future = publisher.publish(topic_path, data)
print("Mensagem publicada! ID:", future.result())

In [ ]:
# Confirmar todas as mensagens pendentes
subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path(PROJECT_ID, SUBSCRIPTION_ID)

try:
    while True:
        # Tentar receber mensagens
        request = {
            "subscription": subscription_path,
            "max_messages": 10,  # Pega até 10 mensagens por vez
            "return_immediately": True
        }
        
        response = subscriber.pull(request=request, timeout=5.0)
        print(f'{len(response.received_messages)} mensages lidas')
        
        if not response.received_messages:
            print("Nenhuma mensagem pendente encontrada.")
            break
        
        # Fazer ack de todas as mensagens recebidas
        ack_ids = [msg.ack_id for msg in response.received_messages]
        

        subscriber.acknowledge(request={"subscription": subscription_path, "ack_ids": ack_ids})
        
    

except Exception as e:
    print(f"Erro ao limpar fila: {e}")